# Imports packages and assign variables

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from IPython.display import display
file_paths = [
    'testData/1.10_A_30s_600ms_3/',
    'testData/2.50_A_30s_600ms_3/',
    'testData/3.90_A_30s_600ms_3/',
    'testData/4.130_A_30s_600ms_3/',
    'testData/5.60_A_30s_200ms_3/',
    'testData/6.60_A_30s_500ms_3/',
    'testData/7.60_A_30s_800ms_3/',
    'testData/8.60_A_30s_1100ms_3/',
    'testData/9.60_A_5s_600ms_3/',
    'testData/10.60_A_20s_600ms_3/',
    'testData/11.60_A_35s_600ms_3/',
    'testData/12.60_A_50s_600ms_3/',
    'testData/13.60_A_30s_600ms_2/',
    'testData/14.60_A_30s_600ms_4/',
    'testData/15.60_A_30s_600ms_6/',
    'testData/16.60_A_30s_600ms_8/',
    'testData/17.60_A_30s_600ms_3/',
    'testData/18.60_A+NA+AZA_30s_600ms_3/',
    'testData/19.60_A+NA_30s_600ms_3/',
    'testData/20.60_A+AZA_30s_600ms_3/',
    'testData/21.10_A+NA+AZA_30s_600ms_3/',
    'testData/22.130_A+NA+AZA_30s_600ms_3/',
    'testData/23.60_A+NA+AZA_30s_200ms_3/',
    'testData/24.60_A+NA+AZA_30s_1100ms_3/',
    'testData/25.60_A+NA+AZA_5s_600ms_3/',
    'testData/26.60_A+NA+AZA_50s_600ms_3/',
    'testData/27.60_A+NA+AZA_30s_600ms_2/',
    'testData/28.60_A+NA+AZA_30s_600ms_8/',
]

labels = [ _[12:-1] if _ [11] == '.' else _[11:-1] for _ in file_paths ]

# Get association rules

In [ ]:
def create_low_high_columns(df, column_names):
    new_df = pd.DataFrame()
    i = 0
    for col in column_names:
        non_zero_values = df[df[col] != 0][col]
        threshold = non_zero_values.median()
        high_col_name = f'high_{col}'
        new_df[high_col_name] = df[col].apply(lambda x: 1 if  x >= threshold else 0).astype(bool)
        i +=1
    return new_df

def association():
  for test_dir in file_paths:
    df_path = f'{test_dir}/stockApp_merged_data.csv'
    df = pd.read_csv(df_path)
    column_names = [
    'application_time', 'database_time', 'api_time', 'application_time_trade', 'database_time_trade','cpu_usage_db_test', 'cpu_usage_db', 'cpu_usage_transaction', 'cpu_usage_web'
]
    df_bin = create_low_high_columns(df, column_names)
    frequent_itemsets = apriori(df_bin, min_support=0.3, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.8)
    important_rules = rules[rules['lift'] > 1.2]
    important_rules = important_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]
    if len(important_rules.index):
        important_rules = important_rules[important_rules['antecedents'].apply(lambda x: len(x) <= 2)]
    if len(important_rules.index):
        important_rules = important_rules[important_rules['consequents'].apply(lambda x: len(x) <= 2)]
    important_rules['support'] = important_rules['support'].round(2)
    important_rules['confidence'] = important_rules['confidence'].round(2)
    important_rules['lift'] = important_rules['lift'].round(2)
    important_rules['antecedents'] = important_rules['antecedents'].apply(lambda x: ', '.join(list(x)))
    important_rules['consequents'] = important_rules['consequents'].apply(lambda x: ', '.join(list(x)))
    print(test_dir[9:-1])
    if len(important_rules.index):
        with pd.option_context('display.max_rows', min(len(important_rules.index),300), 'display.max_columns', 10):
            display(important_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])
    else:
        print('\n')
association()